<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Limit-to-drug-studies-as-of-2010" data-toc-modified-id="Limit-to-drug-studies-as-of-2010-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Limit to drug studies as of 2010</a></span></li><li><span><a href="#Import-study-reference" data-toc-modified-id="Import-study-reference-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import study reference</a></span></li><li><span><a href="#Remove-references-with-only-one-reference-in-the-corpus" data-toc-modified-id="Remove-references-with-only-one-reference-in-the-corpus-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Remove references with only one reference in the corpus</a></span></li><li><span><a href="#Encode-study-nct_id-and-pmid's" data-toc-modified-id="Encode-study-nct_id-and-pmid's-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Encode study nct_id and pmid's</a></span></li><li><span><a href="#Generate-adjacency-matrix-of-nct_id-and-pmid" data-toc-modified-id="Generate-adjacency-matrix-of-nct_id-and-pmid-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generate adjacency matrix of nct_id and pmid</a></span></li></ul></div>

## Limit to drug studies as of 2010

In [7]:
studies = pd.read_sql('select * from studies', con=conn)

In [16]:
lim_regulated_drug = studies['is_fda_regulated_drug']==True
lim_year = pd.to_datetime(studies['start_date']).dt.year>=2010

drug_studies = studies[lim_regulated_drug & lim_year]

## Import study reference

In [51]:
import json
import pandas as pd
import psycopg2

from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np

In [2]:
with open('config.json') as f:
    conf = json.load(f)
conn_str = "host={} dbname={} port={} user={} password={}".format(conf['host'],
                                                                  conf['database'],
                                                                  conf['port'],
                                                                  conf['user'],
                                                                  conf['passw'])
conn = psycopg2.connect(conn_str)

In [44]:
study_references = pd.read_sql('select * from study_references', con=conn)

## Remove references with only one reference in the corpus

In [49]:
# Count number of citations listed in corpus
citation_counts = pd.DataFrame(study_references.groupby('pmid').size()).reset_index()
citation_counts.columns = ['pmid', 'citation_counts']

# Keep citations with more than one reference
multi_citations = citation_counts[citation_counts['citation_counts']>1]

# Keep studies that share a citation with another study
study_references_lim=study_references.merge(multi_citations,on='pmid')

(112897, 6)

## Encode study nct_id and pmid's

In [83]:
study_list = study_references_lim.nct_id.unique()
reference_list = study_references_lim.pmid.unique()

reverse_study_list = {study: int(i) for i, study in enumerate(study_list)}
reverse_reference_list = {ref: int(i) for i, ref in enumerate(reference_list)}

## Generate adjacency matrix of nct_id and pmid

In [87]:
results = np.zeros((study_references_lim.shape[0],multi_citations.shape[0]))

for _, row in study_references_lim.iterrows():
    results[reverse_study_list[row['nct_id']],reverse_reference_list[row['pmid']]]=1

In [86]:
results.shape

(112897, 42038)